In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer, StandardScaler, minmax_scale
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50
pd.options.display.max_rows = 100

In [2]:
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train



In [3]:
df = pd.read_csv('Data_Marketing_Customer_Analysis_Round3.csv')
df.columns



Index(['region', 'customer_lifetime_value', 'response', 'coverage',
       'education', 'effective_to_date', 'month', 'employment_status',
       'gender', 'income', 'location_code', 'marital_status',
       'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size'],
      dtype='object')

In [4]:
numericalX = df.select_dtypes(include=[np.number])
numericalX.head(10)

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809,48029,61,7,52,0,9,292
1,2228,92260,64,3,26,0,1,744
2,14947,22139,100,34,31,0,2,480
3,22332,49078,97,10,3,0,2,484
4,9025,23675,117,33,31,0,7,707
5,4745,50549,61,2,73,0,7,292
6,5035,37405,63,8,99,3,4,287
7,4956,87197,63,35,45,0,3,247
8,5989,66839,154,33,24,0,1,739
9,17114,70961,85,19,8,0,2,408


In [5]:
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train

In [6]:
####### Tuesdays Work##########

In [7]:
#x-y split


X = df.drop('total_claim_amount', axis=1)
y = df['total_claim_amount']



In [8]:
#n/c split

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(np.object)



/var/folders/yc/wft790592j143k8gw6c_18z40000gn/T/ipykernel_5901/96702193.py:4: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categoricalX = X.select_dtypes(np.object)


In [13]:
#train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TT_RATIO, random_state=RAND_STATE)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.head(3)

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
2705,central,4786,no,basic,college,1/2/11,jan,employed,m,45515,...,10,33,0,9,personal auto,personal l3,offer2,call center,four-door car,medsize
2209,central,22445,no,basic,bachelor,2/28/11,feb,unemployed,f,80340,...,32,35,0,2,personal auto,personal l2,offer1,call center,four-door car,medsize
4004,west region,6412,no,extended,bachelor,1/2/11,jan,employed,m,58776,...,20,50,2,4,personal auto,personal l3,offer3,branch,four-door car,medsize


In [10]:
pt = PowerTransformer()


In [11]:
#standardizing the data

ct = ColumnTransformer([("pt", pt, list(numericalX.columns))],
                        remainder='drop',verbose_feature_names_out=True,verbose=True).fit(X_train)
X_train_ct = pd.DataFrame(ct.transform(X_train),columns=ct.get_feature_names_out())
X_test_ct = pd.DataFrame(ct.transform(X_test),columns=ct.get_feature_names_out())



X_train = pd.DataFrame(X_train_ct)
X_test_ct



[ColumnTransformer] ............ (1 of 1) Processing pt, total=   0.1s


In [12]:
#linear regression
X_train_const_ct = sm.add_constant(X_train_ct.to_numpy()) # adding a constant

model = sm.OLS(y_train, X_train_const_ct).fit()
predictions_train = model.predict(X_train_const_ct)

X_test_const_ct = sm.add_constant(X_test_ct) # adding a constant
predictions_test = model.predict(X_test_const_ct)
print_model = model.summary()
print(print_model)



                            OLS Regression Results                            
Dep. Variable:     total_claim_amount   R-squared:                       0.315
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     490.3
Date:                Tue, 31 Jan 2023   Prob (F-statistic):               0.00
Time:                        16:14:12   Log-Likelihood:                -51659.
No. Observations:                7482   AIC:                         1.033e+05
Df Residuals:                    7474   BIC:                         1.034e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        435.8295      2.790    156.232      0.0

In [ ]:
#model interpretation